# Bayesian Statistics for Physicists: 04 Bayesian updating examples

## <a name="Python">Python/Jupyter set up</a>

See <a href="BSFP_01_Overview_and_setup.ipynb">Part 01</a> for overall installation and setup.

In [1]:
%matplotlib inline   

In [2]:
import numpy as np

import scipy.stats as stats
from scipy.stats import norm, uniform

import matplotlib.pyplot as plt
#plt.style.use('seaborn') # pretty matplotlib plots

import corner
import pymc3 as pm


In [3]:
# make font adjustments
#plt.rcParams['font.size'] = 12
#plt.rcParams['legend.fontsize'] = 'medium'
#plt.rcParams['figure.titlesize'] = 'medium'
plt.rcdefaults()  # revert to defaults for now

In [4]:
%%html  
<!-- Use html cell magic to add css styling -->
<style>
  em {
      color: red;
  }
  dd {
      margin-left: 15px;
  }
  .red{color: red}
  .blue{color: blue}
</style>

## <a name="Updating">Bayesian updating examples</a>

### Determining the bias of a coin

The idea here is that we are observing successive flips of a coin, which is a proxy for any process that has a binary outcome.  There is a definite true probability for getting heads, which we'll label $p_h$, but we don't know what it is.  We start with a preconceived notion of the probability expressed in terms of a prior pdf for $p_h$, i.e., $p(p_h)$.  With each flip of the coin, we have more information, so our goal is to <em>update</em> our expectation of $p_h$, meaning we want the posterior $p(p_h\mid \mbox{# tosses, # heads})$. 

In [5]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

import scipy.stats as stats

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Tab, Label, Checkbox, Button
from ipywidgets import FloatSlider, IntSlider, Play, Dropdown, HTMLMath 

from IPython.display import display

# If the coin is fair, prob_heads = 0.5 but you can set it to whatever 
#  you want from 0 to 1.
prob_heads = 0.4

# hyperparameters for several different priors
# prior 1 is uniform in [0,1]
alpha_1 = 1
beta_1 = 1
# prior 2 is concentrated near 0.5 with very small tails
alpha_2 = 30
beta_2 = 30
# prior 3 is peaked at ends, but allows for probability everywhere
alpha_3 = .2
beta_3 = .2

# Calculate Bayesian updating using the conjugate prior for binomial, 
#  which is a beta distribution
dist = stats.beta
n_trials_max = 5000
# heads or tails, 1 or 0, for independent tosses
# mesh for posterior plots; enough to make the pdfs smooth even when narrow
x = np.linspace(0, 1, 301) 

def generate_data(prob_heads=0.5, n_trials_max=5000):
    """
    Generate the distribution for heads or tails, 1 or 0, for independent
    tosses.
    """
    return stats.bernoulli.rvs(prob_heads, size=n_trials_max)  

def update_plot(N=0, jump=1, recalculate_data=True, 
                prob_heads=0.5, n_trials_max=5000):
    """
    Make a new plot based on the current widget settings.
    """
    global data
  
    font_size = 18
    plt.rcParams.update({'font.size': font_size})
    
    fig = plt.figure(figsize=(12,5))
    ax = fig.add_subplot(1, 1, 1)

    if recalculate_data:
        data = generate_data(prob_heads, n_trials_max)
        recalculate_data_w.value = False

    heads = data[:N].sum()   # add up the 1s (= # of heads)
    y_1 = dist.pdf(x, alpha_1 + heads, beta_1 + N - heads)    
    y_2 = dist.pdf(x, alpha_2 + heads, beta_2 + N - heads)   
    y_3 = dist.pdf(x, alpha_3 + heads, beta_3 + N - heads)   
    y_max = np.max([y_1.max(), y_2.max()])  
    # default y_3 distribution has two high max at endpoints for plot
    
    line1, = ax.plot(x, y_1, label="uniform prior", color="blue")
    ax.fill_between(x, 0, y_1, color="blue", alpha=0.1)
    line2, = ax.plot(x, y_2, label="informative prior", color="red")
    ax.fill_between(x, 0, y_2, color="red", alpha=0.1)
    line3, = ax.plot(x, y_3, label="anti prior", color="green")
    ax.fill_between(x, 0, y_3, color="green", alpha=0.1)
     
    ax.set_xlabel("$p_h$, probability of heads") 
    ax.set_yticks([])  # turn off the plotting of ticks on the y-axis
    ax.axvline(prob_heads, 0, 1.1*y_max, color="k", linestyle="--", lw=2)
    ax.annotate("observe {:d} tosses,\n {:d} heads".format(N, heads), 
                xy=(0.05,0.35), xycoords='axes fraction', 
                horizontalalignment='left',verticalalignment='top')
    leg = ax.legend()
    leg.get_frame().set_alpha(0.4)
    ax.autoscale(tight=True)


recalculate_data_w = Checkbox(value=True)    
prob_heads_w = FloatSlider(value=prob_heads, min=0., max=1., step=0.05,
                           description='prob. heads:',
                           continuous_update=False)
n_trials_max_w = IntSlider(value=n_trials_max, min=100, max=10000, step=100,
                           description='max # trials:',
                           continuous_update=False)
 
N_w = IntSlider(value=0, min=0, max=n_trials_max, step=1,
                continuous_update=False)
next_button_w = Button(description='Next', disabled=False,
                       layout=Layout(width='80px'), button_style='', 
                       tooltip='Increment number of trials by jump')
reset_button_w = Button(description='Reset', disabled=False,
                        layout=Layout(width='80px'), button_style='', 
                        tooltip='Reset number of trials to zero')

jump_w = Dropdown(description='Jump:',
                  layout=Layout(width='150px'),
                  options=['1', '10', '100', '1000'],
                  value='1',
                  continuos_update=False,
                  disabled=False,)


def update_N(b):
    """Increment the number of trials N by the Jump value"""
    N_w.value += int(jump_w.value)
    
def reset_N(b):
    """Reset the number of trials N to zero"""
    N_w.value = 0
    
def update_prob_heads(b):
    """Change the value of prob_heads and regenerate data."""
    recalculate_data_w.value = True
    N_w.max = n_trials_max_w.value

next_button_w.on_click(update_N)
reset_button_w.on_click(reset_N)
 
prob_heads_w.observe(update_prob_heads, 'value')    
n_trials_max_w.observe(update_prob_heads, 'value')    


# Text for the help section in HTML (could move this to an external file!)
overview_text = \
   r"""<p>Here we explore Bayesian updating for a coin flip. There is help 
          available under the other tabs.</p>  
          <ul>
            <li>Bayes theorem tab: find out about Bayesian updating.
            <li>Plotting tab: adjust what is plotted and over what intervals.  
                Also about the algorithm used.
            <li>Styling tab: change how the plots look.
            <li>Animate tab: look at animated plots of the time dependence.
          </ul>      
    """ 
Bayes_text = \
    r"""
    <p>Recall Bayes' theorem with $\thetavec$ the vector of parameters 
    we seek and information $I$ is kept implicit.</p>

$$
  \newcommand{\thetavec}{\boldsymbol{\theta}}
  \overbrace{p(\thetavec \mid \textrm{data},I)}^{\textrm{posterior}} =
  \frac{\color{red}{\overbrace{p(\textrm{data} \mid \thetavec,I)}^{\textrm{likelihood}}} \times
   \color{blue}{\overbrace{p(\thetavec \mid I)}^{\textrm{prior}}}}
   {\color{darkgreen}{\underbrace{p(\textrm{data} \mid I)}_{\textrm{evidence}}}}
$$

  <p>If we view the prior as the initial information we have about 
     $\thetavec$, summarized as a probability density function, 
     then Bayes' theorem tells us how to <em>update</em> that 
     information after observing some data: this is the posterior pdf.  
     Here we will look at an example of how this plays out in practice:
     flipping a (biased) coin.</p>     
    """

play_text = \
    r"""
    """

priors_text = \
    r"""
    """

setup_text = \
    r"""
    """

# Widgets for the help section, which are HTMLMath boxes in a Tab widget
help_max_height = '500px'
help_overview_w = HTMLMath(value=overview_text)
help_Bayes_w = HTMLMath(value=Bayes_text)
help_play_w = HTMLMath(value=play_text)
help_priors_w = HTMLMath(value=priors_text)
help_setup_w = HTMLMath(value=setup_text)
help_w = Tab(children=[help_overview_w, help_Bayes_w, help_play_w, 
                       help_priors_w, help_setup_w], 
             layout=Layout(width='95%', max_height=help_max_height))
help_w.set_title(0, 'Overview')
help_w.set_title(1, 'Bayes Theorem')
help_w.set_title(2, 'Play')
help_w.set_title(3, 'Priors')
help_w.set_title(4, 'Set-up')
    

plot_out = widgets.interactive_output(update_plot,
                                      dict(
                                           N=N_w,
                                           jump=jump_w,
                                           recalculate_data=recalculate_data_w,
                                           prob_heads=prob_heads_w,
                                           n_trials_max=n_trials_max_w,
                                      )
                                     )

hbox0 = HBox([next_button_w, jump_w, reset_button_w, prob_heads_w])
hbox1 = HBox([])
hbox2 = HBox([n_trials_max_w])
hbox3 = HBox([help_w])

# We'll set up Tabs to organize the controls.  The Tab contents are declared
#  as tab0, tab1, ... (probably should make this a list?) and the overall Tab
#  is called tab (so its children are tab0, tab1, ...).
tab_height = '70px'  # Fixed minimum height for all tabs. Specify another way?
tab0 = VBox([hbox0], layout=Layout(min_height=tab_height))
tab1 = VBox([hbox1], layout=Layout(min_height=tab_height))
tab2 = VBox([hbox2], layout=Layout(min_height=tab_height))
tab3 = VBox([hbox3], layout=Layout(min_height=tab_height))

tab = Tab(children=[tab0, tab1, tab2, tab3])
tab.set_title(0, 'Play')
tab.set_title(1, 'Priors')
tab.set_title(2, 'Set-up')
tab.set_title(3, 'Help')

UI_box = VBox([tab, plot_out])
display(UI_box)   

### Radioactive decay problem (lighthouse problem)